# Example for calculating the overlap of two histograms

## First we create two samples of random numbers coming from a certain distribution

In [ ]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

We will define 2 random samples coming from 2 different distributions and with different sample size

In [ ]:
sample_1 = np.random.normal(34,8, 1000)
sample_2 = np.random.chisquare(50,500)

In [ ]:
plt.hist(sample_1, alpha=0.5, density=True)
plt.hist(sample_2, alpha=0.5, density=True)
plt.show()

Our goal will be to come up with a measure of how much does this two distributions overlap

## Creating probability densitiy functions

We can use KDE's (Kernel Density Estimate) to get a function that approximates the probability distribtution function of each histogram

In [ ]:
pdf_1 = stats.gaussian_kde(sample_1)
pdf_2 = stats.gaussian_kde(sample_2)

`pdf_1` and `pdf_2` are **functions** that can be evaluated at any point

In [ ]:
print(pdf_1(4.5), pdf_1(40.0), pdf_2(62.3))

In [ ]:
x = np.linspace(0, 100, 1000)
plt.hist(sample_1, alpha=0.5, density=True)
plt.hist(sample_2, alpha=0.5, density=True)
plt.plot(x, pdf_1(x), color='C0')
plt.plot(x, pdf_2(x), color='C1')
plt.show()

Now that `pdf_1` and `pdf_2` are functions we can integrate them with `scipy.integrate.quad`

In [ ]:
import scipy.integrate as integrate

In [ ]:
result = integrate.quad(lambda x: pdf_1(x),  -np.inf,  np.inf)
print(result[0])
result = integrate.quad(lambda x: pdf_2(x),  -np.inf,  np.inf)
print(result[0])

These functions generated by `scipy.stats.gaussian_kde` are already normilized so that
$$ \int f_{\rm kde} (x) dx = 1.$$

However, so that we can measure an overlap we need to normalize the **square** of the function (the way it is done for wavefunctions in quantum mechanics) so that
$$ \int f^2_{\rm kde} (x) dx = 1.$$

Now we can define the overlap as
$$ \int f_{\rm kde} (x) g_{\rm kde} (x) dx$$

## Defining function that calculates the overlap of two functions

In [ ]:
def overlap(f1, f2):
    # First we need to calaculate the normalization constants
    n1 = integrate.quad(lambda x: f1(x)**2,  -np.inf,  np.inf)[0]
    n2 = integrate.quad(lambda x: f2(x)**2,  -np.inf,  np.inf)[0]
    # Normalizing these functions (the quantum mechanis way) means doing the replacement f(x) -> f(x)/sqrt(N)
    # So we calculate the integral of the product and divide by sqrt(N1*N2)
    return integrate.quad(lambda x: f1(x)*f2(x),  -np.inf,  np.inf)[0]/np.sqrt(n1*n2)

### Testing the function by calculating the overlap of a function with itself

In [ ]:
print(overlap(pdf_1, pdf_1))
print(overlap(pdf_2, pdf_2))

### Calculating the overlap of our two kde approximated functions

In [ ]:
print(overlap(pdf_1, pdf_2))

### Making sure that our new overlap operator commutes

In [ ]:
print(overlap(pdf_2, pdf_1))

## Based on our current examples we can define a function that takes two samples and calculates their overlaps

In [ ]:
def sample_overlap(x1, x2):
    # x1 is a numpy array representing the first sample
    # x2 is a numpy array representing the second sample
    pdf1 = stats.gaussian_kde(x1)
    pdf2 = stats.gaussian_kde(x2)
    # We rely on the overlap function we defined earlier
    return overlap(pdf1, pdf2)

Just to confirm that our new function works we test it with the samples we already created and compare results

In [ ]:
# Here we are using the function that takes functions as arguments
print(overlap(pdf_2, pdf_1))
# Here we are using the function that takes samples as arguments
print( sample_overlap(sample_1, sample_2))

## For convinience lets also define a function that plots 2 samples and their KDE's

In [ ]:
def samples_hist_kde_plot(x1, x2, name1, name2, x_axis_name):
    #Finding the minimum and maximum among all elements in both samples
    x_min = min(np.amin(x1), np.amin(x2))
    x_max = max(np.amax(x1), np.amax(x2))
    # Shifting the minimum a little bit to the left
    x_min -= np.abs(x_min)*.1
    # Shifting the maximum a little bit to the right
    x_max += np.abs(x_max)*.1
    # Using those min and max to create an array where the KDE's will be evaluated for plotting
    x = np.linspace(x_min, x_max, 1000)
    pdf1 = stats.gaussian_kde(x1)
    pdf2 = stats.gaussian_kde(x2)
    plt.hist(x1, alpha=0.5, density=True)
    plt.hist(x2, alpha=0.5, density=True)
    plt.plot(x, pdf1(x), color='C0', label = name1)
    plt.plot(x, pdf2(x), color='C1', label = name2)
    plt.xlabel(x_axis_name)
    plt.legend()
    plt.show()

Testing with our current samples

In [ ]:
samples_hist_kde_plot(sample_1, sample_2, 'sample 1', 'sample 2', 'x')

## Now we can try other sampes
## Two samples from gaussian distributions that do not overlap

In [ ]:
sample_3 = np.random.normal(64,4, 1000)
sample_4 = np.random.normal(10,5, 300)
#Notice the different sample sizes
samples_hist_kde_plot(sample_3, sample_4, 'sample 3', 'sample 4', 'x')
print('Samples overlap is: {}'.format(sample_overlap(sample_3, sample_4)))

## Two samples from gaussian distributions that mostly overlap

In [ ]:
sample_5 = np.random.normal(64,4, 1000)
sample_6 = np.random.normal(61,5, 3000)
#Notice the different sample sizes
samples_hist_kde_plot(sample_5, sample_6, 'sample 5', 'sample 6', 'x')
print('Samples overlap is: {}'.format(sample_overlap(sample_5, sample_6)))

## Final note
This is different from the other measure we discussed that calculates the area under where both functions overlap. But that could in principle be defined in a similar fashion. However the the normalization constant and the overlap integral would have to be defined differently.

In fact, here the functions are already normizalized. All that would have to be different is the overlap integral. Not taking the product but the minimum of both fucntions at every value of x

# Testing these new overlap functions with new mass tables

## Reading files with experimental values and calculated values

We are going to read the data files as pandas dataframes

In [ ]:
import pandas as pd

`csv` stands for comma separated file. Our datafile with the experimental is not really in `csv` format, but we can use a a regular expression to say that the separator is an arbitrary number of spaces `\s+` instead of a comma.

We will also rename some columns to use the same names as the theory dataframe. That will make finding intersections easier.

In [ ]:
exp_energies = pd.read_csv('EXPERIMENT_AME2016_headers.dat', sep='\s+')
exp_energies = exp_energies.rename(columns={'Z': 'proton_number', 'N': 'neutron_number', \
                                             'E': 'exp_energy', \
                                             'E/A': 'exp_energy_per_nucleons'})

pandas data frames can be displayed in jupyter notebooks by simply executing a cell with the dataframe's name

In [ ]:
exp_energies

Now we can read a mass table of calculated nuclei with a whole bunch of different properties

In [ ]:
theory = pd.read_csv('ground_states_une0_quartz_intel_mkl-omp.csv')
theory

Now lets plot both nuclear landscape to see where they match and where they don't

In [ ]:
def compare_two_charts(chart1, chart2):
    # chart1 is a pandas dataframe
    # chart2 is a pandas dataframe
    # both charts have to have columns named 'neutron_number' and 'proton_number'
    fig=plt.figure(figsize=(18, 8), dpi= 80, facecolor='w', edgecolor='k')
    plt.scatter(chart1['neutron_number'], chart1['proton_number'], c='C0', s=16)
    plt.scatter(chart2['neutron_number'], chart2['proton_number'], c='C1', s=8)
    plt.show()

In [ ]:
compare_two_charts(exp_energies, theory)

## Creating two dataframes.

### One for the nuclei that match

In [ ]:
def intersect(chart1, chart2):
    return pd.merge(chart1, chart2, how='inner', on=['proton_number', 'neutron_number'])

In [ ]:
exp_match = intersect(exp_energies, theory)
exp_match

### and one for those that are only in the theory set

In [ ]:
theory_only = theory[pd.merge(theory, exp_match, on=['proton_number','neutron_number'], how='left', indicator=True)['_merge'] == 'left_only']

In [ ]:
theory_only

In [ ]:
compare_two_charts(exp_match, theory_only)

Now we can compare the distribution of different DFT properties for the nuclei that have an experimental match and those who don't using the functions we defined previously

In [ ]:
def compare_two_properties(chart1, chart2, column, name1, name2):
    samples_hist_kde_plot(chart1[column], chart2[column], name1, name2, column)
    overlap = sample_overlap(chart1[column], theory_only[column])
    print(column+' overlap is: {}'.format(overlap))
    return overlap

In [ ]:
beta2_total_overlap = compare_two_properties(exp_match, theory_only, 'beta2-deformation_t', 'Theory Experiment match', 'Theory only')

In [ ]:
direct_coulomb = compare_two_properties(exp_match, theory_only, 'direct-coulomb', 'Theory Experiment match', 'Theory only')

There's a simple way to get a list of all the columns names. This should make it easy to go over all columns to get the overlaps

In [ ]:
print(list(theory_only))

## Picking features for the machine learning training:

The ideal feature would have a large overlap so that there's no nuclei where the ML algorithm is being extrapolated, but that also can adopt several different values. A feature that has the same value in all training points is not giving any new information to the ML algorithm